In [18]:
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, joblib
import glob
import pandas as pd
import librosa

In [7]:
train_csv = pd.read_csv('/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data.csv')

In [8]:
train_csv

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.9,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.9,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,85340,AV+PV+TV+MV,Child,Male,105.0,16.6,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
938,85341,AV+PV+TV+MV,Child,Male,92.0,15.2,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
939,85343,AV+PV+TV+MV,Child,Female,97.0,13.5,False,Present,MV+TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
940,85345,AV+PV,Child,Female,132.0,38.1,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN


In [9]:
all_wav_file = glob.glob(data_folder + '*.wav')

In [10]:
len(all_wav_file)

2532

In [11]:
sys.argv

['/home/jh20/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py',
 '-f',
 '/home/jh20/.local/share/jupyter/runtime/kernel-2576eb8c-41e1-4dd2-9249-4a9736e9020e.json']

In [12]:
data_folder = '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/train/'
model_folder = '/home/jh20/narin/physionet/python-classifier-2022/model'

In [13]:
patient_files = find_patient_files(data_folder)
num_patient_files = len(patient_files)

In [14]:
murmur_classes = ['Present', 'Unknown', 'Absent']
num_murmur_classes = len(murmur_classes)
outcome_classes = ['Abnormal', 'Normal']
num_outcome_classes = len(outcome_classes)

In [18]:
os.makedirs(model_folder, exist_ok=True)

In [19]:
features = list()
murmurs = list()
outcomes = list()

In [15]:
current_patient_data = load_patient_data(patient_files[0])
current_recordings = load_recordings(data_folder, current_patient_data)

In [16]:
with open(patient_files[0], 'r') as f:
    data = f.readlines()

In [17]:
data

['2530 4 4000\n',
 'AV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\n',
 'PV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\n',
 'TV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\n',
 'MV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n',
 '#Age: Child\n',
 '#Sex: Female\n',
 '#Height: 98.0\n',
 '#Weight: 15.9\n',
 '#Pregnancy status: False\n',
 '#Murmur: Absent\n',
 '#Murmur locations: nan\n',
 '#Most audible location: nan\n',
 '#Systolic murmur timing: nan\n',
 '#Systolic murmur shape: nan\n',
 '#Systolic murmur grading: nan\n',
 '#Systolic murmur pitch: nan\n',
 '#Systolic murmur quality: nan\n',
 '#Diastolic murmur timing: nan\n',
 '#Diastolic murmur shape: nan\n',
 '#Diastolic murmur grading: nan\n',
 '#Diastolic murmur pitch: nan\n',
 '#Diastolic murmur quality: nan\n',
 '#Outcome: Abnormal\n',
 '#Campaign: CC2015\n',
 '#Additional ID: nan\n']

In [34]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [19]:
current_recordings

[array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16),
 array([2593, 1874, 1370, ..., -637, -255, -243], dtype=int16),
 array([ 2276,  2343,  2448, ..., -2732, -5634, -4290], dtype=int16),
 array([12672, 10041,  3215, ...,  -658,     8,   528], dtype=int16)]

In [23]:
mel = librosa.feature.melspectrogram(  y = current_recordings[0].astype(np.float32) ,sr= 4000, n_mels=128, n_fft=400, hop_length=128, win_length=400)

In [26]:
librosa.power_to_db(mel).shape

(128, 738)

In [26]:
current_recordings[0].shape

(94400,)

In [27]:
current_recordings[1].shape

(52224,)

In [28]:
current_recordings[2].shape

(59968,)

In [29]:
current_recordings[3].shape

(106752,)

In [42]:
current_recordings

[array([-425, 1045,  518, ...,  509,  443,  122], dtype=int16),
 array([2593, 1874, 1370, ..., -637, -255, -243], dtype=int16),
 array([ 2276,  2343,  2448, ..., -2732, -5634, -4290], dtype=int16),
 array([12672, 10041,  3215, ...,  -658,     8,   528], dtype=int16)]

In [ ]:
parser = argparse.ArgumentParser(description = "HMD_trainer")
## Training Settings
parser.add_argument('--num_frames', type=int,   default=200,     help='Duration of the input segments, eg: 200 for 2 second')
parser.add_argument("--lr",default=2e-5, type=float, help="learning rate")
parser.add_argument("--minibatchsize_train", default=64, type=int)
parser.add_argument("--minibatchsize_dev", default=1, type=int)
parser.add_argument("--train_num_workers", default=8, type=int, help="number of training workers")
parser.add_argument("--dev_num_workers", default=1, type=int, help="number of validation workers")




warnings.simplefilter("ignore")
args = parser.parse_args(args=[])

In [33]:
patient_files[0].split('/')[-1].split('.')[0]

'2530'

In [44]:
with open(patient_files[500], 'r') as f:
    data = f.read()

In [45]:
get_patient_id(data)

'68737'

In [46]:
get_num_locations(data)

4

In [47]:
get_locations(data)

['AV', 'PV', 'TV', 'MV']

In [40]:
get_murmur(data)

'Absent'

In [41]:
len(patient_files)

942

In [48]:
num_locations = get_num_locations(data)
recording_information = data.split('\n')[1:num_locations+1]

In [81]:
recording_information

['AV 68737_AV.hea 68737_AV.wav 68737_AV.tsv',
 'PV 68737_PV.hea 68737_PV.wav 68737_PV.tsv',
 'TV 68737_TV.hea 68737_TV.wav 68737_TV.tsv',
 'MV 68737_MV.hea 68737_MV.wav 68737_MV.tsv']

In [109]:
import random
import numpy as np


recordings = {}

for i in range(4):
    entries = recording_information[i].split(' ')
    recording_file = entries[2]
    location = entries[0]
    filename = os.path.join(data_folder, recording_file)
    recording, frequency = load_wav_file(filename)
    
    
    length = 20*4000
    if recording.shape[0] <= length:
        shortage = length - recording.shape[0]
        recording = np.pad(recording, (0, shortage), 'wrap')
    start_frame = np.int64(random.random()*(recording.shape[0]-length))
    recording = recording[start_frame:start_frame + length] 
    recordings[location] = recording.astype(np.float32)

In [82]:
recording, frequency = load_wav_file(filename)

In [110]:
recordings

{'AV': array([1891., 1860.,  505., ..., -243., -261., -164.], dtype=float32),
 'PV': array([ -356.,  -423.,    99., ..., -1927., -1922., -1903.], dtype=float32),
 'TV': array([715., 669., 773., ...,  32.,   8., 119.], dtype=float32),
 'MV': array([-414., -258., -179., ...,   10.,   19.,  142.], dtype=float32)}

In [ ]:
for k,v in recordings.items():
    log_mel = take_log(mel_spec(torch.from_numpy(v)))
    log_mel = log_mel.unsqueeze(0)
    
    

In [93]:
len(recordings)

4

In [119]:
recordings

{'AV': array([1891., 1860.,  505., ..., -243., -261., -164.], dtype=float32),
 'PV': array([ -356.,  -423.,    99., ..., -1927., -1922., -1903.], dtype=float32),
 'TV': array([715., 669., 773., ...,  32.,   8., 119.], dtype=float32),
 'MV': array([-414., -258., -179., ...,   10.,   19.,  142.], dtype=float32)}

In [129]:
num_location= len(recordings)
current_feature = np.zeros((num_location, 80000))
i = 0
for k,v in recordings.items():
    current_feature[i,:] = v
    i+=1

In [126]:
current_feature = np.zeros((num_location, 80000))

In [145]:
current_feature

array([[ 1891.,  1860.,   505., ...,  -243.,  -261.,  -164.],
       [ -356.,  -423.,    99., ..., -1927., -1922., -1903.],
       [  715.,   669.,   773., ...,    32.,     8.,   119.],
       [ -414.,  -258.,  -179., ...,    10.,    19.,   142.]])

In [154]:
np.mean(current_feature, axis=0).astype(np.float32)

array([ 459. ,  462. ,  299.5, ..., -532. , -539. , -451.5], dtype=float32)

In [155]:
recordings['MV']

array([-414., -258., -179., ...,   10.,   19.,  142.], dtype=float32)

In [95]:
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram
mel_spec = MelSpectrogram(
            sample_rate=4000,
            n_fft=400,
            win_length=400,
            hop_length=128,
            f_min=0,
            n_mels=128,
            window_fn=torch.hamming_window,
            wkwargs={"periodic": False},
            power=1,
        )

In [195]:
mel = mel_spec(torch.from_numpy(recordings['AV'].astype(np.float32)))

In [106]:
def take_log(mels):

    amp_to_db = AmplitudeToDB(stype="amplitude")
    amp_to_db.amin = 1e-5  
    return amp_to_db(mels)

In [196]:
take_log(mel)

tensor([[80.3810, 62.7282, 66.8454,  ..., 71.9923, 58.5625, 68.6145],
        [80.9390, 72.3045, 68.8841,  ..., 78.1348, 71.0171, 72.4136],
        [81.6487, 77.5446, 70.9289,  ..., 82.3820, 76.7865, 75.6011],
        ...,
        [61.3785, 53.9913, 48.7594,  ..., 43.2049, 44.6559, 47.4078],
        [59.8013, 51.9922, 43.7277,  ..., 41.0759, 42.5014, 42.4430],
        [58.0107, 51.9606, 45.5729,  ..., 37.6095, 38.8560, 38.5879]])

In [111]:
get_age(data)

'Child'

In [53]:
import torchaudio

audio, sr = torchaudio.load(filename)

/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [169]:
current_murmur = np.zeros(num_murmur_classes, dtype=np.float32)
murmur = get_murmur(current_patient_data)
if murmur in murmur_classes:
    j = murmur_classes.index(murmur)
    current_murmur[j] = 1


current_outcome = np.zeros(num_outcome_classes, dtype=np.float32)
outcome = get_outcome(current_patient_data)
if outcome in outcome_classes:
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1


In [157]:
current_murmur = np.zeros(num_murmur_classes, dtype=np.float32)

In [163]:
murmur = get_murmur(data)

In [170]:
outcomes

[array([0., 1.]),
 array([0., 1.], dtype=float32),
 array([0., 1.], dtype=float32)]

In [112]:
num_location_list = []
for filename in patient_files:
    with open(filename, 'r') as f:
        current_patient_data = f.read()
    num_location = get_num_locations(current_patient_data)
    num_location_list.append(num_location)

In [115]:
max(num_location_list)

6

In [116]:
min(num_location_list)

1

In [118]:
get_pregnancy_status(data)

False

In [140]:
len(recordings.keys())

4

In [141]:
dic = {'a': 12}

In [144]:
list(dic.keys())[0]

'a'

In [183]:
examples = {}
for filename in patient_files:
    with open(filename, 'r') as f:
        current_patient_data = f.read()
    
    if filename.split('/')[-1].split('.')[0] not in examples.keys():
        current_patient = patient_files[0].split('/')[-1].split('.')[0]
        examples[current_patient] = {
            "filename": filename,
            "num_location": get_num_locations(current_patient_data),
            "frequency" : get_frequency(current_patient_data),
            "location" : get_locations(current_patient_data),
            "age" : get_age(current_patient_data),
            "sex" : get_sex(current_patient_data),
            "height" : get_height(current_patient_data),
            "weight" : get_weight(current_patient_data),
            "pregnancy_status" : get_pregnancy_status(current_patient_data),
            "murmur" : get_murmur(current_patient_data),
            "outcome" : get_outcome(current_patient_data)
               
        }

In [184]:
examples

{'2530': {'filename': '/home/jh20/Data/nr_data/ECG/Physionet2022/physionet.org/files/circor-heart-sound/1.0.3/training_data/2530.txt',
  'num_location': 4,
  'frequency': 4000.0,
  'location': ['AV', 'PV', 'TV', 'MV'],
  'age': 'Child',
  'sex': 'Female',
  'height': 98.0,
  'weight': 15.9,
  'pregnancy_status': False,
  'murmur': 'Absent',
  'outcome': 'Abnormal'}}

In [338]:
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/jh20/narin/physionet/python-classifier-2022")
from helper_code import *
import torchaudio
import random
import numpy as np
from torchaudio.transforms import AmplitudeToDB, MelSpectrogram


def to_mono(recordings, random_ch=False):
    
    num_location = len(recordings)

    if num_location > 1:  # multi channel
        if not random_ch:
            current_feature = np.zeros((num_location, 80000))
            i = 0
            for k,v in recordings.items():
                current_feature[i,:] = v
                i+=1
            current_feature = np.mean(current_feature, axis=0).astype(np.float32)
            
    else:
        one_loc = list(recordings.keys())[0]
        current_feature = recordings[one_loc].astype(np.float32)
        
    return current_feature    
    

def read_recordings(data_folder, data):
    num_locations = get_num_locations(data)
    recording_information = data.split('\n')[1:num_locations+1]

    recordings = {}

    for i in range(num_locations):
        entries = recording_information[i].split(' ')
        recording_file = entries[2]
        location = entries[0]
        filename = os.path.join(data_folder, recording_file)
        recording, frequency = load_wav_file(filename)


        length = 20*4000
        if recording.shape[0] <= length:
            shortage = length - recording.shape[0]
            recording = np.pad(recording, (0, shortage), 'wrap')
        start_frame = np.int64(random.random()*(recording.shape[0]-length))
        recording = recording[start_frame:start_frame + length] 
        recordings[location] = recording.astype(np.float32)

    return recordings


def take_log(mels):

    amp_to_db = AmplitudeToDB(stype="amplitude")
    amp_to_db.amin = 1e-5  
    return amp_to_db(mels)



class train_loader(Dataset):
    def __init__(self,data_folder, **kwargs):
        self.data_folder = data_folder
        self.mel_spec = MelSpectrogram(
            sample_rate=4000,
            n_fft=400,
            win_length=400,
            hop_length=128,
            f_min=0,
            n_mels=128,
            window_fn=torch.hamming_window,
            wkwargs={"periodic": False},
            power=1,
        )

        
        
        patient_files = find_patient_files(data_folder)
        num_patient_files = len(patient_files)
        
        murmur_classes = ['Present', 'Unknown', 'Absent']
        num_murmur_classes = len(murmur_classes)
        outcome_classes = ['Abnormal', 'Normal']
        num_outcome_classes = len(outcome_classes)
        
        
        examples = {}
        for filename in patient_files:
            with open(filename, 'r') as f:
                current_patient_data = f.read()
            
            if filename.split('/')[-1].split('.')[0] not in examples.keys():
                current_patient = filename.split('/')[-1].split('.')[0]
                examples[current_patient] = {
                    "filename": filename,
                    "num_location": get_num_locations(current_patient_data),
                    "frequency" : get_frequency(current_patient_data),
                    "location" : get_locations(current_patient_data),
                    "age" : get_age(current_patient_data),
                    "sex" : get_sex(current_patient_data),
                    "height" : get_height(current_patient_data),
                    "weight" : get_weight(current_patient_data),
                    "pregnancy_status" : get_pregnancy_status(current_patient_data),
                    "murmur" : get_murmur(current_patient_data),
                    "outcome" : get_outcome(current_patient_data)
                       
                }
        self.examples = examples
        self.examples_list = list(examples.keys())
        
        
        if len(self.examples_list) != len(patient_files):
            raise ValueError('length is different!')
    
        
    def __getitem__(self, index):
        
        # log mel feature
        current_ex_dict = self.examples[self.examples_list[index]]
        current_data = load_patient_data(current_ex_dict["filename"])
        current_recordings = read_recordings(data_folder, current_data)
        
        current_recordings = to_mono(current_recordings, random_ch=False)
        
#         current_recordings = torch.tensor(current_recordings, dtype=torch.float32).clone().detach()
        
        log_mel_feature = take_log(mel_spec(torch.from_numpy(current_recordings)))
        
        data_feature = torch.tensor(log_mel_feature, dtype=torch.float32, requires_grad=F)
        # label
        
        current_murmur = torch.zeros(num_murmur_classes, dtype=torch.float32)
        murmur = get_murmur(current_data)
        if murmur in murmur_classes:
            j = murmur_classes.index(murmur)
            current_murmur[j] = 1
        murmur_label =  torch.as_tensor(current_murmur, dtype=torch.float32, requires_grad=True)

        current_outcome = torch.zeros(num_outcome_classes, dtype=torch.float32)
        outcome = get_outcome(current_data)
        if outcome in outcome_classes:
            j = outcome_classes.index(outcome)
            current_outcome[j] = 1
        outcome_label = torch.tensor(current_outcome, dtype=torch.float32, requires_grad=True)    
        
        return data_feature, murmur_label, outcome_label
    def __len__(self):
        return len(self.examples_list)

In [ ]:
# def myCollateFn(sample_batch):
#     sample_batch = sorted(sample_batch, key=lambda x: x[0].shape[0], reverse=True)
#     log_mel_feature = [torch.tensor(x[0]) for x in sample_batch]
#     current_murmur = [torch.tensor(x[1]) for x in sample_batch]
#     current_outcome = [torch.tensor(x[2]) for x in sample_batch]

#     return log_mel_feature, current_murmur, current_outcome

# class myDataLoader(DataLoader):
#     def __init__(self, *args, **kwargs):
#         super(myDataLoader, self).__init__(*args, **kwargs)
#         self.collate_fn = myCollateFn

In [292]:
from torch.utils.data import Dataset,DataLoader , RandomSampler

In [340]:
point_sampler = RandomSampler(trainset)

In [339]:
trainset = train_loader(data_folder)

In [264]:
trainset[0]

(tensor([[93.5905, 84.8715, 54.2976,  ..., 61.4996, 76.1757, 68.7957],
         [93.2550, 85.1750, 61.1994,  ..., 70.4779, 78.1291, 64.5556],
         [92.9053, 85.6067, 65.7122,  ..., 75.5789, 80.1079, 52.0486],
         ...,
         [41.9424, 45.5729, 46.1398,  ..., 48.5557, 52.0632, 52.0810],
         [40.7872, 46.2383, 46.4158,  ..., 48.4979, 46.0868, 48.9658],
         [33.4897, 44.9085, 46.0755,  ..., 46.2945, 41.5224, 46.3135]]),
 tensor([0., 0., 1.]),
 tensor([1., 0.]))

In [282]:
audio, murmur, outcome = next(iter(trainloader))

In [288]:
point_sampler = RandomSampler(map_dataset)

NameError: name 'RandomSampler' is not defined

In [341]:
dataloader_train = myDataLoader(dataset=trainset,
                        batch_size=32,
                               sampler =point_sampler,
                               num_workers=6)

In [342]:
a, b, c = next(iter(dataloader_train))

/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/ipykernel_launcher.py:134: UserWarning: To copy construct from a tensor, it is recommended 

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-338-2462352190a6>", line 142, in __getitem__
    murmur_label =  torch.as_tensor(current_murmur, dtype=torch.float32, requires_grad=True)
TypeError: as_tensor() got an unexpected keyword argument 'requires_grad'


In [287]:
a

[tensor([[68.6683, 75.1568, 74.4430,  ..., 74.3135, 75.4665, 72.1969],
         [63.5838, 76.4039, 76.3497,  ..., 78.9415, 79.1801, 76.7821],
         [33.9292, 77.7890, 78.2919,  ..., 82.5523, 82.3204, 80.3725],
         ...,
         [41.5171, 46.6283, 48.5438,  ..., 56.9770, 60.9018, 50.3342],
         [39.3141, 48.5832, 49.6623,  ..., 56.1821, 60.3242, 48.3023],
         [41.4528, 51.9555, 48.8688,  ..., 56.1280, 60.8780, 49.9353]]),
 tensor([[73.8207, 74.6053, 71.2825,  ..., 81.1578, 81.7733, 78.9637],
         [75.7456, 74.4874, 76.5250,  ..., 81.7968, 81.5786, 75.2309],
         [77.7021, 74.4182, 80.4133,  ..., 82.5917, 81.4121, 66.1206],
         ...,
         [61.0917, 52.4004, 49.3426,  ..., 48.4785, 50.8986, 57.2339],
         [60.5735, 52.3945, 47.9301,  ..., 51.6270, 54.1112, 57.9481],
         [59.2988, 49.4595, 45.2043,  ..., 50.5741, 49.6934, 52.9861]]),
 tensor([[80.2390, 77.4743, 77.5777,  ..., 73.4035, 67.9269, 69.5096],
         [79.1911, 74.8747, 77.0203,  ..., 72

In [272]:
a.shape

torch.Size([32, 128, 626])

In [273]:
b

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])

In [21]:
for i in range(num_patient_files):

    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings = load_recordings(data_folder, current_patient_data)

    # Extract features.
    current_features = get_patient_id(current_patient_data, current_recordings)
    features.append(current_features)

    # Extract labels and use one-hot encoding.
    current_murmur = np.zeros(num_murmur_classes, dtype=int)
    murmur = get_murmur(current_patient_data)
    if murmur in murmur_classes:
        j = murmur_classes.index(murmur)
        current_murmur[j] = 1
    murmurs.append(current_murmur)

    current_outcome = np.zeros(num_outcome_classes, dtype=int)
    outcome = get_outcome(current_patient_data)
    if outcome in outcome_classes:
        j = outcome_classes.index(outcome)
        current_outcome[j] = 1
    outcomes.append(current_outcome)

features = np.vstack(features)
murmurs = np.vstack(murmurs)
outcomes = np.vstack(outcomes)

NameError: name 'get_features' is not defined

In [ ]:
def train_challenge_model(data_folder, model_folder, verbose):
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting features and labels from the Challenge data...')

    murmur_classes = ['Present', 'Unknown', 'Absent']
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)

    features = list()
    murmurs = list()
    outcomes = list()

    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        # Load the current patient data and recordings.
        current_patient_data = load_patient_data(patient_files[i])
        current_recordings = load_recordings(data_folder, current_patient_data)

        # Extract features.
        current_features = get_features(current_patient_data, current_recordings)
        features.append(current_features)

        # Extract labels and use one-hot encoding.
        current_murmur = np.zeros(num_murmur_classes, dtype=int)
        murmur = get_murmur(current_patient_data)
        if murmur in murmur_classes:
            j = murmur_classes.index(murmur)
            current_murmur[j] = 1
        murmurs.append(current_murmur)

        current_outcome = np.zeros(num_outcome_classes, dtype=int)
        outcome = get_outcome(current_patient_data)
        if outcome in outcome_classes:
            j = outcome_classes.index(outcome)
            current_outcome[j] = 1
        outcomes.append(current_outcome)

    features = np.vstack(features)
    murmurs = np.vstack(murmurs)
    outcomes = np.vstack(outcomes)

    # Train the model.
    if verbose >= 1:
        print('Training model...')

    # Define parameters for random forest classifier.
    n_estimators   = 123  # Number of trees in the forest.
    max_leaf_nodes = 45   # Maximum number of leaf nodes in each tree.
    random_state   = 6789 # Random state; set for reproducibility.

    imputer = SimpleImputer().fit(features)
    features = imputer.transform(features)
    murmur_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, murmurs)
    outcome_classifier = RandomForestClassifier(n_estimators=n_estimators, max_leaf_nodes=max_leaf_nodes, random_state=random_state).fit(features, outcomes)

    # Save the model.
    save_challenge_model(model_folder, imputer, murmur_classes, murmur_classifier, outcome_classes, outcome_classifier)

    if verbose >= 1:
        print('Done.')

In [358]:
90000// 80000

1

In [359]:
100000//80000

1

In [362]:
import librosa

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f6958dafcb0>
Traceback (most recent call last):
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/multiprocessing/connection.py", line 921, in wait
    ready = selector.select(timeout)
  File "/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/selectors.py", line 415, in select
    fd_event_list = self.

In [363]:
import sys
sys.argv

['/home/jh20/anaconda3/envs/ECAPA/lib/python3.7/site-packages/ipykernel_launcher.py',
 '-f',
 '/home/jh20/.local/share/jupyter/runtime/kernel-988a635e-7b60-4334-bbbd-92f7856c231a.json']

In [ ]:
feature.melspectrogram( y = sr= 4000, n_mels=128, n_fft=400, hop_length=128, win_length=400)